In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
url = "https://www.nykaa.com/wellness/sports-nutrition/c/18696"

In [3]:
header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"}

In [5]:
resp = requests.get(url,headers=header)
print(resp)

<Response [200]>


In [6]:
soup = BeautifulSoup(resp.content)
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1.0,maximum-scale=5.0" name="viewport"/><meta content="#fff" name="theme-color"/><meta content="NykaaFrontendTeam" name="author"/><link href="//images-static.naikaa.com/media/wysiwyg/uiTools/2024-3/beauty.png" rel="icon" type="image/x-icon"/><title data-react-helmet="true">Shop For Authentic Sports Nutrition Products At Best Deals &amp; Offers</title><meta content="Shop For Authentic Sports Nutrition Products At Best Deals &amp; Offers" data-react-helmet="true" property="og:title"/><meta content="Shop For Authentic Sports Nutrition Products At Best Deals &amp; Offers" data-react-helmet="true" property="twitter:title"/><meta content="Buy truly genuine Sports Nutrition Products from Nykaa at the best deals. Order now with easy returns &amp; fast delivery. Explore a wide collection of Sports Nutrition Products for all body types!" data-react-helmet="true" name="description"/><meta 

In [7]:
container = soup.find_all("div", class_ = "css-ifdzs8")

In [8]:
first_product = container[0]
first_product

<div class="css-ifdzs8"><style data-emotion-css="d5z3ro">.css-d5z3ro{padding:30px 10.18%;position:relative;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;height:100%;}</style><div class="css-d5z3ro"><style data-emotion-css="wkluxr">.css-wkluxr{margin:14px;position:absolute;top:0px;left:0;z-index:1;text-transform:uppercase;font-weight:600;}</style><div class="css-wkluxr"><style data-emotion-css="ycjyr6">.css-ycjyr6{margin-left:0;padding-left:0;list-style:none;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;margin:0 0 0.5rem;}.css-ycjyr6 li{font-size:12px;font-weight:500;line-height:16px;-webkit-letter-spacing:0;-moz-letter-spacing:0;-ms-letter-spacing:0;letter-spacing:0;margin-right:0.25rem;border:none;display:inline-block;padding:0 0.5rem;}.css-ycjyr6 li.featured{co

In [11]:
first_product.find('div', class_="css-xrzmfa").text.strip()

'Wellbeing Nutrition Whey Protein Isolate, 31g Protein, 4B CF...'

In [12]:
first_product.find("span",class_="css-111z9ua").text.strip()

'₹4749'

In [11]:
first_product.find("span",class_="css-17x46n5").text.strip()

'MRP:₹4999'

In [12]:
first_product.find("span",class_="css-cjd9an").text.strip()

'5% Off'

In [15]:
first_product.find("li", class_="custom-tag").text.strip()

'AD'

In [13]:
second_product = container[1]

In [16]:
rating =second_product.find("span", class_="css-1qbvrhp").text.strip()
rating.replace("(", "").replace(")", "").strip()

'881'

In [17]:
second_product.find("div",class_="css-ijlamg").text.strip()

'5 sizes'

## Health Products

In [76]:
import requests
from bs4 import BeautifulSoup
import time

# lists for data
product_title_s = []
price_s = []
rating_s = []
offer_s = []
original_price_s = []
tags_s = []
sizes_s = []
stock_s = []

# session with headers
session = requests.Session()
session.headers.update({
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9"
})

for pgs in range(1, 55):  
    url = f"https://www.nykaa.com/wellness/sports-nutrition/c/18696?page_no={pgs}&sort=popularity&eq=desktop"

    try:
        response = session.get(url, timeout=15, allow_redirects=False)
        response.raise_for_status()
    except Exception as e:
        print(f"⚠️ Skipping page {pgs} due to error: {e}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    container = soup.find_all("div", class_="css-ifdzs8")

    for products in container:
        # product title
        product_title_1 = products.find("div", class_="css-xrzmfa")
        product_title_s.append(product_title_1.text.strip() if product_title_1 else "NaN")

        # price
        price_1 = products.find("span", class_="css-111z9ua")
        price_value = price_1.text.strip() if price_1 else "NaN"
        price_s.append(price_value)

        # rating
        rating_element = products.find("span", class_="css-1qbvrhp")
        if rating_element:
            rating_1 = rating_element.text.strip()
            rating_1 = rating_1.replace("(", "").replace(")", "").strip()
            rating_s.append(rating_1)
        else:
            rating_s.append("NaN")

        # offer + original price
        offer_element = products.find("span", class_="css-cjd9an")
        orignal_price_1 = products.find("span", class_="css-17x46n5")

        if offer_element:  # when discount exists
            offer_s.append(offer_element.text.strip().replace(" Off", ""))
            original_price_s.append(orignal_price_1.text.strip() if orignal_price_1 else "NaN")
        else:  # no discount → set original_price same as current price
            offer_s.append("NaN")
            original_price_s.append(price_value)

        # tags (FEATURED / BESTSELLER)
        tags = products.find_all("li", class_="custom-tag")
        if tags:
            tag_list = [t.text.strip() for t in tags]
            tags_s.append(", ".join(tag_list))   # join multiple tags
        else:
            tags_s.append("NaN")

        #sizes
        sizes_1 = products.find("div", class_="css-ijlamg")
        sizes_s.append(sizes_1.text.strip() if sizes_1 else "NaN")

        #stock
        stock_1 = products.find("span", class_="css-lg5xc9")
        stock_s.append(stock_1.text.strip() if stock_1 else "In Stock")

    # pause to avoid blocking
    time.sleep(2)

print("✅ Scraping completed!")
print("Products collected:", len(product_title_s))


✅ Scraping completed!
Products collected: 1060


In [77]:
## create a data frame
health_product_dict = {
'Product_Name' : product_title_s, 
'Original_Price' : original_price_s, 
'Offer_Percentage' : offer_s, 
'Final_Price' : price_s,
'Reviews' : rating_s, 
'Tags'  : tags_s,
'Sizes' : sizes_s,
'Stock' : stock_s
}
health_products_df = pd.DataFrame(health_product_dict)
health_products_df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes,Stock
0,"Wellbeing Nutrition Whey Protein Isolate, 31g ...",MRP:₹4999,5%,₹4749,NaN,"AD, BESTSELLER",NaN,In Stock
1,Optimum Nutrition (ON) Gold Standard 100% Whey...,MRP:₹9299,20%,₹7399,881,BESTSELLER,5 sizes,In Stock
2,"OZiva Protein & Herbs Women(with Shatavari, Gi...",MRP:₹1699,6%,₹1599,494,BESTSELLER,2 sizes,In Stock
3,Isopure Low Carb Whey Protein Isolate Powder -...,MRP:₹5949,20%,₹4749,114,BESTSELLER,2 sizes,In Stock
4,Wellbeing Nutrition Organic Vegan Protein Isol...,MRP:₹1999,5%,₹1899,2,"AD, BESTSELLER",NaN,In Stock
...,...,...,...,...,...,...,...,...
1055,Nutrabay Gold Isolate Whey Protein Powder - Do...,MRP:₹7899,29%,₹5608,NaN,NaN,NaN,Out Of Stock
1056,MYPRO SPORT NUTRITION Premium BCAA'S With 5G O...,₹1399,NaN,₹1399,NaN,NaN,NaN,Out Of Stock
1057,"HealthyHey Sports BCAA Powder 2:1:1, Branched ...",MRP:₹1699,44%,₹949,2,NaN,NaN,Out Of Stock
1058,MYPRO SPORT NUTRITION Premium BCAA'S With 5G O...,₹1399,NaN,₹1399,1,NaN,NaN,Out Of Stock


In [78]:
health_products_df.to_csv("health_products.csv", index=False)

## Baby Products

In [72]:
import requests
from bs4 import BeautifulSoup
import time

# lists for data
product_title_s = []
price_s = []
rating_s = []
offer_s = []
original_price_s = []
tags_s = []
sizes_s = []
stock_s = []

# session with headers
session = requests.Session()
session.headers.update({
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9"
})

for pgs in range(1, 40):  
    url = f"https://www.nykaa.com/mom-baby/baby-care/c/14798?page_no={pgs}&sort=popularity&dir=desc&id=14798&order=popularity&ptype=lst&redirectpath=slug&root=nav_2&eq=desktop"

    try:
        response = session.get(url, timeout=15, allow_redirects=False)
        response.raise_for_status()
    except Exception as e:
        print(f"⚠️ Skipping page {pgs} due to error: {e}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    container = soup.find_all("div", class_="css-ifdzs8")

    for products in container:
        # product title
        product_title_1 = products.find("div", class_="css-xrzmfa")
        product_title_s.append(product_title_1.text.strip() if product_title_1 else "NaN")

        # price
        price_1 = products.find("span", class_="css-111z9ua")
        price_value = price_1.text.strip() if price_1 else "NaN"
        price_s.append(price_value)

        # rating
        rating_element = products.find("span", class_="css-1qbvrhp")
        if rating_element:
            rating_1 = rating_element.text.strip()
            rating_1 = rating_1.replace("(", "").replace(")", "").strip()
            rating_s.append(rating_1)
        else:
            rating_s.append("NaN")

        # offer + original price
        offer_element = products.find("span", class_="css-cjd9an")
        orignal_price_1 = products.find("span", class_="css-17x46n5")

        if offer_element:  # when discount exists
            offer_s.append(offer_element.text.strip().replace(" Off", ""))
            original_price_s.append(orignal_price_1.text.strip() if orignal_price_1 else "NaN")
        else:  # no discount → set original_price same as current price
            offer_s.append("NaN")
            original_price_s.append(price_value)

        # tags (FEATURED / BESTSELLER)
        tags = products.find_all("li", class_="custom-tag")
        if tags:
            tag_list = [t.text.strip() for t in tags]
            tags_s.append(", ".join(tag_list))   # join multiple tags
        else:
            tags_s.append("NaN")

        #sizes
        sizes_1 = products.find("div", class_="css-ijlamg")
        sizes_s.append(sizes_1.text.strip() if sizes_1 else "NaN")

        #stock
        stock_1 = products.find("span", class_="css-lg5xc9")
        stock_s.append(stock_1.text.strip() if stock_1 else "In Stock")
        
    # pause to avoid blocking
    time.sleep(2)

print("✅ Scraping completed!")
print("Products collected:", len(product_title_s))


✅ Scraping completed!
Products collected: 728


In [73]:
## create a data frame
baby_product_dict = {
'Product_Name' : product_title_s, 
'Original_Price' : original_price_s, 
'Offer_Percentage' : offer_s, 
'Final_Price' : price_s,
'Reviews' : rating_s, 
'Tags'  : tags_s,
'Sizes' : sizes_s,
'Stock' : stock_s
}
baby_products_df = pd.DataFrame(baby_product_dict)
baby_products_df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes,Stock
0,Orimii Bump Hydrating Body Butter for Reducing...,MRP:₹695,5%,₹660,77,"AD, BESTSELLER",NaN,In Stock
1,Bioderma Atoderm Creme Ultra-Nourising Moistur...,₹699,NaN,₹699,15435,BESTSELLER,NaN,In Stock
2,Max Care Virgin Coconut Oil Cold Pressed,₹325,NaN,₹325,22862,BESTSELLER,3 sizes,In Stock
3,Cetaphil Baby Daily Lotion With Shea Butter Vi...,MRP:₹950,7%,₹884,5142,BESTSELLER,NaN,In Stock
4,Nat Habit Baby Body Lotion - Almond Date,MRP:₹591,30%,₹414,17,AD,2 sizes,In Stock
...,...,...,...,...,...,...,...,...
723,Puddles Kids Unicorn Magic Organic Mist,₹499,NaN,₹499,NaN,NaN,NaN,Out Of Stock
724,Mylo Care Cold Pressed Extra Virgin Coconut Oi...,₹349,NaN,₹349,45,NaN,1 size,Out Of Stock
725,TuCo Intelligent Kids Tear-Free Kids Anti Dand...,MRP:₹900,46%,₹491,NaN,NaN,NaN,Out Of Stock
726,TIKITORO Teens Nourishing Bath Bar,₹310,NaN,₹310,1,NaN,NaN,Out Of Stock


In [74]:
baby_products_df.to_csv("baby_products.csv", index=False)

## Beauty Products

In [67]:
import requests
from bs4 import BeautifulSoup
import time

# lists for data
product_title_s = []
price_s = []
rating_s = []
offer_s = []
original_price_s = []
tags_s = []
sizes_s = []
stock_s = []

# session with headers
session = requests.Session()
session.headers.update({
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9"
})

for pgs in range(1, 90):  
    url = f"https://www.nykaa.com/luxe/makeup/c/1101?page_no={pgs}&sort=popularity&eq=desktop"

    try:
        response = session.get(url, timeout=15, allow_redirects=False)
        response.raise_for_status()
    except Exception as e:
        print(f"⚠️ Skipping page {pgs} due to error: {e}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    container = soup.find_all("div", class_="css-ifdzs8")

    for products in container:
        # product title
        product_title_1 = products.find("div", class_="css-xrzmfa")
        product_title_s.append(product_title_1.text.strip() if product_title_1 else "NaN")

        # price
        price_1 = products.find("span", class_="css-111z9ua")
        price_value = price_1.text.strip() if price_1 else "NaN"
        price_s.append(price_value)

        # rating
        rating_element = products.find("span", class_="css-1qbvrhp")
        if rating_element:
            rating_1 = rating_element.text.strip()
            rating_1 = rating_1.replace("(", "").replace(")", "").strip()
            rating_s.append(rating_1)
        else:
            rating_s.append("NaN")

        # offer + original price
        offer_element = products.find("span", class_="css-cjd9an")
        orignal_price_1 = products.find("span", class_="css-17x46n5")

        if offer_element:  # when discount exists
            offer_s.append(offer_element.text.strip().replace(" Off", ""))
            original_price_s.append(orignal_price_1.text.strip() if orignal_price_1 else "NaN")
        else:  # no discount → set original_price same as current price
            offer_s.append("NaN")
            original_price_s.append(price_value)

        # tags (FEATURED / BESTSELLER)
        tags = products.find_all("li", class_="custom-tag")
        if tags:
            tag_list = [t.text.strip() for t in tags]
            tags_s.append(", ".join(tag_list))   # join multiple tags
        else:
            tags_s.append("NaN")

         #sizes
        sizes_1 = products.find("div", class_="css-ijlamg")
        sizes_s.append(sizes_1.text.strip() if sizes_1 else "NaN")

        #stock
        stock_1 = products.find("span", class_="css-lg5xc9")
        stock_s.append(stock_1.text.strip() if stock_1 else "In Stock")

    # pause to avoid blocking
    time.sleep(2)

print("✅ Scraping completed!")
print("Products collected:", len(product_title_s))

✅ Scraping completed!
Products collected: 1691


In [68]:
## create a data frame
beauty_product_dict = {
'Product_Name' : product_title_s, 
'Original_Price' : original_price_s, 
'Offer_Percentage' : offer_s, 
'Final_Price' : price_s,
'Reviews' : rating_s, 
'Tags'  : tags_s,
'Sizes' : sizes_s,
'Stock' : stock_s
}
beauty_products_df = pd.DataFrame(beauty_product_dict)
beauty_products_df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes,Stock
0,O.P.I RapiDry Quick-Dry Nail Polish,MRP:₹550,12%,₹484,33,AD,25 shades,In Stock
1,Charlotte Tilbury Airbrush Flawless Foundation,MRP:₹5100,30%,₹3570,3921,BESTSELLER,8 shades,In Stock
2,M.A.C MACximal Matte Mini Lipstick,MRP:₹1650,30%,₹1150,33054,BESTSELLER,14 shades,In Stock
3,LANEIGE Lip Glowy Balm,₹750,NaN,₹750,21822,BESTSELLER,6 shades,In Stock
4,Too Faced Better Than Sex Liquid Eye Liner,₹2350,NaN,₹2350,959,AD,NaN,In Stock
...,...,...,...,...,...,...,...,...
1686,IDUN Minerals Eye & Lip Definer Brush,₹1300,NaN,₹1300,NaN,NaN,NaN,In Stock
1687,IDUN Minerals Concealer Brush,₹1300,NaN,₹1300,NaN,NaN,NaN,In Stock
1688,IDUN Minerals Eyeshadow Brush,₹1300,NaN,₹1300,NaN,NaN,NaN,In Stock
1689,IDUN Minerals Blending Brush,₹1300,NaN,₹1300,NaN,NaN,NaN,In Stock


In [30]:
beauty_products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product_Name      1699 non-null   object
 1   Original_Price    1699 non-null   object
 2   Offer_Percentage  1699 non-null   object
 3   Final_Price       1699 non-null   object
 4   Reviews           1699 non-null   object
 5   Tags              1699 non-null   object
 6   Sizes             1699 non-null   object
dtypes: object(7)
memory usage: 93.0+ KB


## EDA - DATA CLEANING

In [69]:
# Save DataFrame to CSV
beauty_products_df.to_csv("beauty_products.csv", index=False) 

In [41]:
df = pd.read_csv("beauty_products.csv")
df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes
0,Pixi On The Glow Cream Blush,₹1950,NaN,₹1950,7606.0,"AD, BESTSELLER",3 shades
1,Charlotte Tilbury Airbrush Flawless Foundation,₹5100,NaN,₹5100,3921.0,BESTSELLER,8 shades
2,M.A.C MACximal Matte Mini Lipstick,MRP:₹1650,30%,₹1150,33054.0,BESTSELLER,14 shades
3,LANEIGE Lip Glowy Balm,₹750,NaN,₹750,21822.0,BESTSELLER,6 shades
4,Too Faced Better Than Sex Liquid Eye Liner,₹2350,NaN,₹2350,959.0,AD,NaN
...,...,...,...,...,...,...,...
1694,IDUN Minerals Eye & Lip Definer Brush,₹1300,NaN,₹1300,NaN,NaN,NaN
1695,IDUN Minerals Concealer Brush,₹1300,NaN,₹1300,NaN,NaN,NaN
1696,IDUN Minerals Eyeshadow Brush,₹1300,NaN,₹1300,NaN,NaN,NaN
1697,IDUN Minerals Blending Brush,₹1300,NaN,₹1300,NaN,NaN,NaN


Original_Price - removing MRP:₹

In [42]:
df["Original_Price"]=df["Original_Price"].astype(str).str.replace("MRP:", "").str.replace("₹", "").str.strip()

Final_Price - removing ₹ symbol

In [43]:
df["Final_Price"] = df["Final_Price"].astype(str).str.replace("₹", "").str.strip()

sizes - removing shades

In [44]:
df["Sizes"] = df["Sizes"].astype(str).str.replace("shades", "").str.strip()

Offer - (remove % sign)

In [45]:
df["Offer_Percentage"] = df["Offer_Percentage"].astype(str).str.replace("%", "").str.strip()

In [46]:
df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes
0,Pixi On The Glow Cream Blush,1950,nan,1950,7606.0,"AD, BESTSELLER",3
1,Charlotte Tilbury Airbrush Flawless Foundation,5100,nan,5100,3921.0,BESTSELLER,8
2,M.A.C MACximal Matte Mini Lipstick,1650,30,1150,33054.0,BESTSELLER,14
3,LANEIGE Lip Glowy Balm,750,nan,750,21822.0,BESTSELLER,6
4,Too Faced Better Than Sex Liquid Eye Liner,2350,nan,2350,959.0,AD,nan
...,...,...,...,...,...,...,...
1694,IDUN Minerals Eye & Lip Definer Brush,1300,nan,1300,NaN,NaN,nan
1695,IDUN Minerals Concealer Brush,1300,nan,1300,NaN,NaN,nan
1696,IDUN Minerals Eyeshadow Brush,1300,nan,1300,NaN,NaN,nan
1697,IDUN Minerals Blending Brush,1300,nan,1300,NaN,NaN,nan


converting into numeric - Original_Price,Final_Price,Sizes,Offer_Percentage

In [47]:
df["Original_Price"] = pd.to_numeric(df["Original_Price"], errors="coerce")

In [48]:
df["Final_Price"] = pd.to_numeric(df["Final_Price"], errors="coerce")

In [49]:
df["Sizes"] = pd.to_numeric(df["Sizes"], errors="coerce")

In [50]:
df["Offer_Percentage"] = pd.to_numeric(df["Offer_Percentage"], errors="coerce")

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_Name      1699 non-null   object 
 1   Original_Price    1699 non-null   int64  
 2   Offer_Percentage  289 non-null    float64
 3   Final_Price       1699 non-null   int64  
 4   Reviews           1340 non-null   float64
 5   Tags              186 non-null    object 
 6   Sizes             743 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 93.0+ KB


In [52]:
df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes
0,Pixi On The Glow Cream Blush,1950,NaN,1950,7606.0,"AD, BESTSELLER",3.0
1,Charlotte Tilbury Airbrush Flawless Foundation,5100,NaN,5100,3921.0,BESTSELLER,8.0
2,M.A.C MACximal Matte Mini Lipstick,1650,30.0,1150,33054.0,BESTSELLER,14.0
3,LANEIGE Lip Glowy Balm,750,NaN,750,21822.0,BESTSELLER,6.0
4,Too Faced Better Than Sex Liquid Eye Liner,2350,NaN,2350,959.0,AD,NaN
...,...,...,...,...,...,...,...
1694,IDUN Minerals Eye & Lip Definer Brush,1300,NaN,1300,NaN,NaN,NaN
1695,IDUN Minerals Concealer Brush,1300,NaN,1300,NaN,NaN,NaN
1696,IDUN Minerals Eyeshadow Brush,1300,NaN,1300,NaN,NaN,NaN
1697,IDUN Minerals Blending Brush,1300,NaN,1300,NaN,NaN,NaN


filling values

In [53]:
df["Sizes"] = df["Sizes"].fillna(1).astype(int)

0 (zero fill) → Treats missing reviews as no reviews.
Best when NaN means "not reviewed yet".

In [54]:
df["Reviews"] = df["Reviews"].fillna(0).astype(int)

In [55]:
df.isna().sum()

Product_Name           0
Original_Price         0
Offer_Percentage    1410
Final_Price            0
Reviews                0
Tags                1513
Sizes                  0
dtype: int64

In [56]:
df["Offer_Percentage"] = df["Offer_Percentage"].fillna(0).astype(int)

In [57]:
df["Tags"] = df["Tags"].fillna("Unknown")

In [58]:
df.isna().sum()

Product_Name        0
Original_Price      0
Offer_Percentage    0
Final_Price         0
Reviews             0
Tags                0
Sizes               0
dtype: int64

In [59]:
df

,Product_Name,Original_Price,Offer_Percentage,Final_Price,Reviews,Tags,Sizes
0,Pixi On The Glow Cream Blush,1950,0,1950,7606,"AD, BESTSELLER",3
1,Charlotte Tilbury Airbrush Flawless Foundation,5100,0,5100,3921,BESTSELLER,8
2,M.A.C MACximal Matte Mini Lipstick,1650,30,1150,33054,BESTSELLER,14
3,LANEIGE Lip Glowy Balm,750,0,750,21822,BESTSELLER,6
4,Too Faced Better Than Sex Liquid Eye Liner,2350,0,2350,959,AD,1
...,...,...,...,...,...,...,...
1694,IDUN Minerals Eye & Lip Definer Brush,1300,0,1300,0,Unknown,1
1695,IDUN Minerals Concealer Brush,1300,0,1300,0,Unknown,1
1696,IDUN Minerals Eyeshadow Brush,1300,0,1300,0,Unknown,1
1697,IDUN Minerals Blending Brush,1300,0,1300,0,Unknown,1


In [ ]:
df1 = pd.read_csv("beauty_products.csv")
df1